pyJHTDB are failed to compile on windows. One alternative way might be to use [zeep](https://python-zeep.readthedocs.io/en/master/) package.    
More details can be found at [http://turbulence.pha.jhu.edu/service/turbulence.asmx](http://turbulence.pha.jhu.edu/service/turbulence.asmx)

In [1]:
import zeep
import numpy as np

client = zeep.Client('http://turbulence.pha.jhu.edu/service/turbulence.asmx?WSDL')
ArrayOfFloat = client.get_type('ns0:ArrayOfFloat')
ArrayOfArrayOfFloat = client.get_type('ns0:ArrayOfArrayOfFloat')
SpatialInterpolation = client.get_type('ns0:SpatialInterpolation')
TemporalInterpolation = client.get_type('ns0:TemporalInterpolation')

token="edu.jhu.pha.turbulence.testing-201406" #replace with your own token

nnp=5 #number of points
points=np.random.rand(nnp,3)

# convert to JHTDB structures
x_coor=ArrayOfFloat(points[:,0].tolist())
y_coor=ArrayOfFloat(points[:,1].tolist())
z_coor=ArrayOfFloat(points[:,2].tolist())
point=ArrayOfArrayOfFloat([x_coor,y_coor,z_coor]);

print(points)

[[0.53187028 0.03233128 0.36918436]
 [0.71432427 0.59386035 0.82498249]
 [0.81072237 0.83556546 0.1506142 ]
 [0.60134585 0.71154697 0.31154439]
 [0.07116765 0.99824415 0.24781622]]


#### In `GetData_Python`, Function_name could be  

GetVelocity,            GetMagneticField,          GetVectorPotential,    
GetVelocityGradient,    GetMagneticFieldGradient,  GetVectorPotentialGradient,    
GetVelocityHessian,     GetMagneticHessian,        GetVectorPotentialHessian,    
GetVelocityLaplacian,   GetMagneticFieldLaplacian, GetVectorPotentialLaplacian,    

GetPressure,            GetTemperature,            GetDensity,    
GetPressureGradient,    GetTemperatureGradient,    GetDensityGradient,    
GetPressureHessian,     GetTemperatureHessian,     GetDensityHessian,     

GetVelocityAndPressure, GetVelocityAndTemperature, GetForce, GetInvariant

In [2]:
Function_name="GetVelocityGradient" 
time=0.6
number_of_component=9 # change this based on function_name, see http://turbulence.pha.jhu.edu/webquery/query.aspx

result=client.service.GetData_Python(Function_name, token,"isotropic1024coarse", 0.6, 
                                     SpatialInterpolation("None_Fd4"), TemporalInterpolation("None"), point)
result=np.array(result).reshape((-1, number_of_component))

print(result)

[[ -0.56701374  -7.902108    10.5850477    2.26401734  -3.15606952
    5.209894    -1.8741504   -0.34069368   3.555669  ]
 [  6.15157461   1.84039414  -4.72127676   5.623206    -4.64100456
    7.90722561  -2.551727    -4.98853445  -1.3864001 ]
 [ -1.67070842  -7.99403954  -5.926757    23.4961185    1.03677046
   29.9265633   -6.925942   -19.5277824    0.5717542 ]
 [  9.975634     3.51199937  23.3713722   -5.68434429   0.17755659
  -17.7729378   -2.3305788   -9.197428   -10.353199  ]
 [  2.91647      2.37627721  -3.24783945  -4.913837    -1.907366
   -2.58240175   6.87442827   3.72034025  -1.09677577]]


#### In `GetPosition_Python`, Function_name could be
GetPosition only.

In [3]:
Function_name="GetPosition"
startTime=0.1
endTime=0.2
dt=0.02
number_of_component=3 # change this based on function_name, see http://turbulence.pha.jhu.edu/webquery/query.aspx

result=client.service.GetPosition_Python(Function_name, token,"isotropic1024coarse", startTime, endTime, dt, 
                                         SpatialInterpolation("None"), point)
result=np.array(result).reshape((-1, number_of_component))
print(result)

[[ 0.5509413  -0.11838733  0.3403707 ]
 [ 0.7480935   0.5429849   0.73117405]
 [ 0.7962904   0.65870273  0.05734132]
 [ 0.558507    0.56875074  0.29501018]
 [ 0.01652166  0.9107007   0.18408126]]


#### In `GetFilter_Python`, Function_name could be    

GetBoxFilter, GetBoxFilterSGSscalar, GetBoxFilterSGSvector,     
GetBoxFilterSGSsymtensor, GetBoxFilterSGStensor, GetBoxFilterGradient.

In [4]:
Function_name="GetBoxFilter" #could also be
field="u"
time=0.6
filterwidth=0.05
spacing=0 #spacing is only used in GetBoxFilterGradient, but always provide it.
number_of_component=3 # change this based on function_name, see http://turbulence.pha.jhu.edu/webquery/query.aspx

result=client.service.GetFilter_Python("GetBoxFilter",token,"isotropic1024coarse", field, 
                                       time, filterwidth, SpatialInterpolation("None"), point, spacing) 
result=np.array(result).reshape((-1, number_of_component))
print(result)

[[ 0.27206102 -1.30481255  0.06045016]
 [-0.12582369 -0.37959147 -1.10447919]
 [-0.8601785  -0.65143985 -0.8783631 ]
 [-0.48609233 -0.95902133 -0.22834729]
 [-0.42911473 -0.80377346  0.25465983]]


In [5]:
import struct
import base64

field="u"
timestep=1
x_start=1
y_start=1
z_start=1
x_end=2
y_end=5
z_end=8
x_step=1
y_step=1
z_step=1
filter_width=0
    
result=client.service.GetAnyCutoutWeb(token,"isotropic1024coarse", field, timestep,
                                       x_start, y_start, z_start, x_end, y_end, z_end,
                                      x_step, y_step, z_step, filter_width, "")  # put empty string for the last parameter

# transfer base64 format to numpy
number_of_component=3 # change this based on the field
nx=len(range(x_start, x_end+1, x_step))
ny=len(range(y_start, y_end+1, y_step))
nz=len(range(z_start, z_end+1, z_step))
base64_len=int(nx*ny*nz*number_of_component)
base64_format='<'+str(base64_len)+'f'

result=struct.unpack(base64_format, result)
result=np.array(result).reshape((nz, ny, nx, number_of_component))
print(result.shape)  # see the shape of the result and compare it with nx, ny, nz and number of component

(20, 20, 20, 3)
